In [9]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, label_binarize
from sklearn.metrics import r2_score, make_scorer, roc_auc_score
from sklearn.neural_network import MLPClassifier
import warnings
warnings.filterwarnings('ignore')
import optuna
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay,classification_report
from sklearn.tree import DecisionTreeClassifier

In [2]:
df2=pd.read_csv(r"D:\Default.csv")
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   default  10000 non-null  object 
 1   student  10000 non-null  object 
 2   balance  10000 non-null  float64
 3   income   10000 non-null  float64
dtypes: float64(2), object(2)
memory usage: 312.6+ KB


In [3]:
stu=(df2["student"]=="Yes").astype(int)
x1=df2["balance"].values.reshape(-1,1)
x3=stu.values.reshape(-1,1)
x2=df2["income"].values.reshape(-1,1)
y=df2["default"]== "Yes"
# Matriz de predictores
X3=np.hstack([x1,x2,x3])

modelo=LogisticRegression()
modelo.fit(X3,y)

#Intercepto y coeficientes
beta0_3=modelo.intercept_
beta1_3=modelo.coef_

print("Intercept: ",beta0_3)
print("Coeficientes: ",beta1_3)

Intercept:  [-10.90180794]
Coeficientes:  [[ 5.73061102e-03  3.96163654e-06 -6.12572751e-01]]


In [ ]:
df2['student_Yes'] = pd.get_dummies(df2['student'], drop_first=True)
X3 = df2[['balance', 'income', 'student_Yes']]
y= df2['default']=='Yes'
logis = LogisticRegression()
logis.fit(X3, y)
logis.score(X3,y)

0.9732

In [ ]:
clasi=DecisionTreeClassifier()

1.0

In [ ]:
from sklearn.utils import resample
X_boot, y_boot = resample(
    X3, 
    y, 
    replace=True,         
    n_samples=len(X3),   
    random_state=42   
)



In [13]:
n_bootstraps = 500
all_cols = X3.columns
pred_samples = np.zeros((n_bootstraps, len(y)))
selected_cols_list = []
models = []
for i in range(n_bootstraps):

    selected_cols = np.random.choice(all_cols, size=2, replace=False)
    selected_cols_list.append(selected_cols)
    idx = np.random.choice(len(y), size=len(y), replace=True)
    X_sample = X3.iloc[idx][selected_cols]
    y_sample = y[idx]

    model = DecisionTreeClassifier()
    model.fit(X_sample, y_sample)
    models.append(model)

for i in range(n_bootstraps):
    selected_cols = selected_cols_list[i]
    pred_samples[i, :] = models[i].predict_proba(X3[selected_cols])[:, 1]

bagged_pred = pred_samples.mean(axis=0)
auc_bagging = roc_auc_score(y, bagged_pred)

print("AUC del Bagging:",auc_bagging)

AUC del Bagging: 1.0
